In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
import shapely
#can use the constructive operation make_valid to check and fix invalid geometries
import leafmap as leafmap
import dash_leaflet as dl #from tutorial, replace with leafmap later
#dcc = dash_core_components
#hmtl = dash_html_components
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import json


ModuleNotFoundError: No module named 'dash_leaflet'

In [ ]:
practice_csv = pd.read_csv('data/terrawise/terrawise-practice-info.csv', usecols=['label','Year', 'Land use', 'Urea', 'Lime','Synthetic_fertiliser','Biochar'])
practice_csv=practice_csv[practice_csv['label']>0]

#use this version of the csv for the dropdown menu select:
practice_csv_drop_year= practice_csv.drop(['Year'], axis=1)

geom_gdf = gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geom_gdf.to_crs("WGS84", inplace=True)
geom2_gdf = gpd.read_file('data/terrawise/Stratification/2/cea part/cea_2.shp')
geom2_gdf.to_crs("WGS84", inplace=True)
geom3_gdf = gpd.read_file('data/terrawise/Stratification/3/cea part/cea_3.shp')
geom3_gdf.to_crs("WGS84", inplace=True)


In [ ]:
input_geom=[geom_gdf, geom2_gdf, geom3_gdf]
output_geom=[]

#loop to attach practice data to each CEA
for i,geom in enumerate (input_geom):
    #print("STARTING GEOM: \n",geom)
    cea=i+1  #ideally this is programmatic, but for now setting as 1 so index starts right
    frame = geom.dissolve()
    frame['label']=cea
    frame_pracice_data = practice_csv[practice_csv['label']==cea]
    geom_combined = frame.set_index('label').join(frame_pracice_data.set_index('label'))
    output_geom.append(geom_combined)

#print(output_geom[0])

# test = output_geom[0]
# test=test[test['Year']==2017]
# print(test)

# test2=practice_csv[practice_csv['Year']==2017]
# print(test2)

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

controls = dbc.Card(
    [html.Div(
            [dbc.Label("Select management practice"),
                dcc.Dropdown(practice_csv_drop_year.columns, id='candidate'),]),],
    body=True,
)

controls_year = dbc.Card(
    [html.Div(
            [dbc.Label("Year"),
                dcc.Slider(practice_csv['Year'].min(),
                            practice_csv['Year'].max(),
                            step=None,
                            value=practice_csv['Year'].min(),
                            marks={str(year): str(year) for year in practice_csv['Year'].unique()},
                            id='year-slider'
				),
                ]
            ),
    ],
    body=True,
)

In [ ]:
#have both dash and dash bootstrap
app.layout = dbc.Container(
    [
        html.H1("Practice management by CEA"),
        html.Hr(),
        dbc.Row([dbc.Col(dcc.Graph(id="map"), md=7),
                 dbc.Col(dcc.Graph(id="graph"), md=3)
                ]),
        dbc.Row([dbc.Col(controls, md=3),
                 dbc.Col(controls_year, md=7)
                 ],),
    ],
    fluid=True,
)


@app.callback(
    [Output("map", "figure"),
     Output("graph", "figure")], 
    [Input('year-slider', 'value'),
     Input('candidate', 'value')])


def display_choropleth(selected_year, candidate):
    data=practice_csv[practice_csv['Year']==selected_year]
    graph = px.histogram(data, x=candidate, y='label', histfunc='count', color=candidate)
    graph.update_layout()

    yearly_geom=[]
    #use the input year to create subset of polygons absed on year attribute
    for i,geom in enumerate (output_geom):
        frame = geom[geom['Year']==selected_year]
        yearly_geom.append(frame)

    #use the yearly subset of polygons to then select management practices
    geodf = pd.concat([yearly_geom[0], yearly_geom[1], yearly_geom[2]])
    fig = px.choropleth_mapbox(
        data_frame=geodf.set_index('FID'),
        geojson=geodf.geometry,
        locations=geodf.index,
        mapbox_style = 'open-street-map',
        center=dict(lat=-33.69, lon=118.2),
        zoom=10,
        color=candidate
    )
    fig.update_layout()
    return fig, graph

if __name__ == '__main__':
    app.run_server(debug=False, port=8051)
